# Урок 5. Рекуррентные нейронные сети

ЗАДАНИЕ 1

Попробуйте изменить параметры нейронной сети работающей с датасетом imdb так, чтобы улучшить ее точность. Приложите анализ.

In [5]:
from __future__ import print_function

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 50 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Построение модели...
Процесс обучения...


TypeError: Can not convert a NoneType into a Tensor or Operation.

In [6]:
import tensorflow as tf
print(tf.__version__)
print(keras.__version__)

2.2.0
2.3.1


**Не могу сделать д з к 5 уроку - ошибка в коде при обучении модели на imdb:**

model.fit(x_train, y_train,
batch_size=batch_size,
epochs=1, 
validation_data=(x_test, y_test))

TypeError: Can not convert a NoneType into a Tensor or Operation.

ЗАДАНИЕ 2.

Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [7]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 1 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию 
с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/1
158773/158773 [==============================] - 17s 106us/step - loss: 6092.8262
Генерация из посева:  on, turni
 on, turnia h hh  h  o o  oa oaaaaaa  o       oaaaaaaaaa          oaaaaaaaaa          oaaaaaaaaa          oaaa


In [10]:
def exploring_model(batch_size,epochs,RNN_type):
    model = Sequential()
    if RNN_type == 'LSTM':
        model.add(LSTM(HIDDEN_SIZE,
                       return_sequences=False,
                       input_shape=(SEQLEN, nb_chars),
                       dropout=0.2,
                       recurrent_dropout=0.2,
                       unroll=True))
    elif RNN_type == 'GRU':
        model.add(
            GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
                HIDDEN_SIZE,
                return_sequences=False,
                input_shape=(SEQLEN, nb_chars),
                unroll=True
            )
        )
    model.add(Dense(nb_chars))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

    print('batch_size=',batch_size)
    print('epochs=',epochs)
    print('RNN_type=',RNN_type)
    
    # выполнение серий тренировочных и демонстрационных итераций 
    for iteration in range(NUM_ITERATIONS):

        # для каждой итерации запуск передачи данных в модель 
        print("=" * 50)
        print("Итерация #: %d" % (iteration))
        model.fit(X, y, batch_size=batch_size, epochs=epochs)

        # Select a random example input sequence.
        test_idx = np.random.randint(len(input_chars))
        test_chars = input_chars[test_idx]

        # для числа шагов предсказаний использование текущей тренируемой модели 
        # конструирование one-hot encoding для тестирования input и добавление предсказания.
        print("Генерация из посева: %s" % (test_chars))
        print(test_chars, end="")
        for i in range(NUM_PREDS_PER_EPOCH):

            # здесь one-hot encoding.
            X_test = np.zeros((1, SEQLEN, nb_chars))
            for j, ch in enumerate(test_chars):
                X_test[0, j, char2index[ch]] = 1

            # осуществление предсказания с помощью текущей модели.
            pred = model.predict(X_test, verbose=0)[0]
            y_pred = index2char[np.argmax(pred)]

            # вывод предсказания добавленного к тестовому примеру 
            print(y_pred, end="")

            # инкрементация тестового примера содержащего предсказание
            test_chars = test_chars[1:] + y_pred
    return y_pred

In [11]:
NUM_ITERATIONS=10

batch_sizes=[128,256,512]
epochses=[1,5,10]
RNN_types=['GRU','LSTM']
for batch_size in batch_sizes:
    for epochs in epochses:
        for RNN_type in RNN_types:
            exploring_model(batch_size,epochs,RNN_type)

batch_size= 128
epochs= 1
RNN_type= GRU
Итерация #: 0
Epoch 1/1
158773/158773 [==============================] - 14s 88us/step - loss: 6039.0366
Генерация из посева: ir that wo
ir that wonnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn==================================================
Итерация #: 1
Epoch 1/1
158773/158773 [==============================] - 14s 86us/step - loss: 5873.7484
Генерация из посева: mallest id
mallest id    e                                                                                               ==================================================
Итерация #: 2
Epoch 1/1
158773/158773 [==============================] - 13s 84us/step - loss: 5984.0764
Генерация из посева:  --mystery
 --mysteryd :d                                                                                                ==================================================
Итерация #: 3
Epoch 1/1
158773/158773 [===========================

158773/158773 [==============================] - 14s 90us/step - loss: 5792.5889
Epoch 2/5
158773/158773 [==============================] - 14s 90us/step - loss: 6179.2454
Epoch 3/5
158773/158773 [==============================] - 15s 91us/step - loss: 6111.1787
Epoch 4/5
158773/158773 [==============================] - 14s 90us/step - loss: 5781.7734
Epoch 5/5
158773/158773 [==============================] - 14s 87us/step - loss: 6241.4653
Генерация из посева: y of room!
y of room!   i aoa a a   i iaia    a o i  aai i oi       o    oa   i  oa   i  oa   i  oa   i  oa   i  oa   i  ==================================================
Итерация #: 4
Epoch 1/5
158773/158773 [==============================] - 15s 92us/step - loss: 5597.2582
Epoch 2/5
158773/158773 [==============================] - 14s 88us/step - loss: 5975.8239
Epoch 3/5
158773/158773 [==============================] - 14s 90us/step - loss: 6275.1568
Epoch 4/5
158773/158773 [==============================] - 14s 90us/step - 

158773/158773 [==============================] - 16s 104us/step - loss: 3791.2512
Epoch 4/5
158773/158773 [==============================] - 15s 93us/step - loss: 3870.2108
Epoch 5/5
158773/158773 [==============================] - 15s 95us/step - loss: 3784.4503
Генерация из посева: y and sadl
y and sadltttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt==================================================
Итерация #: 6
Epoch 1/5
158773/158773 [==============================] - 23s 143us/step - loss: 3860.7576
Epoch 2/5
158773/158773 [==============================] - 20s 127us/step - loss: 4085.7548
Epoch 3/5
158773/158773 [==============================] - 15s 92us/step - loss: 3971.4778
Epoch 4/5
158773/158773 [==============================] - 15s 94us/step - loss: 3898.1346
Epoch 5/5
158773/158773 [==============================] - 15s 93us/step - loss: 3782.5895
Генерация из посева:  just thin
 just thini   in                        

158773/158773 [==============================] - 15s 96us/step - loss: 6121.2865
Epoch 7/10
158773/158773 [==============================] - 15s 96us/step - loss: 6296.1576
Epoch 8/10
158773/158773 [==============================] - 15s 96us/step - loss: 6149.6351
Epoch 9/10
158773/158773 [==============================] - 15s 94us/step - loss: 5696.3326
Epoch 10/10
158773/158773 [==============================] - 15s 95us/step - loss: 6113.0871
Генерация из посева: voice of t
voice of ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt==================================================
Итерация #: 5
Epoch 1/10
158773/158773 [==============================] - 15s 96us/step - loss: 6301.3629
Epoch 2/10
158773/158773 [==============================] - 15s 95us/step - loss: 6305.5135
Epoch 3/10
158773/158773 [==============================] - 15s 95us/step - loss: 6118.8497
Epoch 4/10
158773/158773 [==============================] - 15s 92u

158773/158773 [==============================] - 17s 108us/step - loss: 3781.0796
Epoch 9/10
158773/158773 [==============================] - 18s 111us/step - loss: 3781.3246
Epoch 10/10
158773/158773 [==============================] - 16s 100us/step - loss: 3902.7255
Генерация из посева: the cat in
the cat intttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt==================================================
Итерация #: 2
Epoch 1/10
158773/158773 [==============================] - 17s 105us/step - loss: 3880.2959
Epoch 2/10
158773/158773 [==============================] - 16s 99us/step - loss: 3780.1173
Epoch 3/10
158773/158773 [==============================] - 18s 116us/step - loss: 3960.3508
Epoch 4/10
158773/158773 [==============================] - 18s 110us/step - loss: 3885.9228
Epoch 5/10
158773/158773 [==============================] - 15s 97us/step - loss: 3758.6870
Epoch 6/10
158773/158773 [==============================] - 1

KeyboardInterrupt: 

**Вывод:
лучше распознает последовательность сеть с типом='LSTM' и epochs=10 и batch_sizes=128**